In [5]:
import numpy as np
import pandas as pd 
import math
import matplotlib.pyplot as plt
import time
import os
src_path = "../Src/"
data_path = "../Measurement/"
cora_path = "../CORA-Z7/"
import sys
sys.path.insert(1, src_path)
sys.path.insert(2, cora_path)

In [28]:
#g_ULTX
def F5288_2x2(f,gain_code, phicut, rx_theta, info_show=False):
    # f: frequenct (GHz)
    dfs = pd.DataFrame();theta=rx_theta;phi=phicut
    rawdatapath=data_path+"F5288_BVH_p"+str(phi)+"t0g"+str(gain_code)
    file_path = f"{rawdatapath}/Phi{phi}_theta_{theta}.csv"
    df = pd.read_csv(file_path)
    dfs = pd.concat([dfs, df['S21_LogM']], axis=1)
    dfs = pd.concat([dfs, df['S31_LogM']], axis=1)
    idx=(f-24)/0.01
    if 27.5<f<28.5:
        compensate_gain= 57.13
        gain= dfs['S21_LogM'][idx]+compensate_gain
    if 28.5<f<29.5:
        compensate_gain= 56.74
        gain= dfs['S21_LogM'][idx]+compensate_gain
    
    if 29.5<f<30.5:
        compensate_gain= 57.82
        gain= dfs['S21_LogM'][idx]+compensate_gain

    if info_show == True:
        print("F5288 2x2 BCH Measurement info List as below")
        print("Setup in 7F D Chamber , OTA Distance: 1.7 m")
        print("compensate_gain:"+str(round(compensate_gain,2))+"dB")
        print("Polarization: RHCP beamform to theta = 0 deg.")
        print("measured S21:"+str(round(dfs['S21_LogM'][idx],2))+"dB")
        print("measured S31:"+str(round(dfs['S31_LogM'][idx],2))+"dB")
        print("F5288_2x2 gain = S21 + compensate_gain:"+str(round(gain,2))+"dB")
        XPD=dfs['S21_LogM'][idx]-dfs['S31_LogM'][idx];print("XPD:"+str(round(XPD,2))+"dB")
        AR=20*np.log10(abs((10**(XPD/20))+1)/abs((10**(XPD/20))-1));print("AR:"+str(round(AR,2))+"dB")

    return round(gain,2)

# g_DLRX
def F6212_2x2(f,gain_code, phicut, tx_theta, info_show=False):
    # f: frequenct (GHz)
    dfs = pd.DataFrame();theta=tx_theta;phi=phicut
    rawdatapath=data_path+"F6212_BVH_p"+str(phi)+"t0g"+str(gain_code)
    file_path = f"{rawdatapath}/Phi0_theta_{theta}.csv"
    df = pd.read_csv(file_path)
    dfs = pd.concat([dfs, df['S12_LogM']], axis=1)
    dfs = pd.concat([dfs, df['S13_LogM']], axis=1)
    idx=(f-15)/0.01
    if 16.5<f<17.5:
        compensate_gain= 51.29
        gain= dfs['S12_LogM'][idx]+compensate_gain
    if 17.5<f<18.5:
        compensate_gain= 50.44
        gain= dfs['S12_LogM'][idx]+compensate_gain
    
    if 18.5<f<19.5:
        compensate_gain= 51.58
        gain= dfs['S12_LogM'][idx]+compensate_gain
        

    if info_show == True:
        print("F6212 2x2 BCH Measurement info List as below")
        print("Setup in 7F D Chamber , OTA Distance: 1.7 m") 
        print("Polarization: RHCP beamform to theta = 0 deg.")
        print("measured S12:"+str(round(dfs['S12_LogM'][idx],2))+"dB")
        print("measured S13:"+str(round(dfs['S13_LogM'][idx],2))+"dB")
        print("F6212_2x2 gain = S21 + compensate_gain:"+str(round(gain,2))+"dB")
        print("compensate_gain:"+str(round(compensate_gain,2))+"dB")
        XPD=dfs['S12_LogM'][idx]-dfs['S13_LogM'][idx];print("XPD:"+str(round(XPD,2))+"dB")
        AR=20*np.log10(abs((10**(XPD/20))+1)/abs((10**(XPD/20))-1));print("AR:"+str(round(AR,2))+"dB")
        
    return round(gain,2)
def LPhorn():
    return 23
def CPLTX_1x4():
    return 22
def CPLRX_1x4():
    return 22
def cubesat_antenna_TXant_v3_ch1(f):
    if 17.5<f<18.5:
        return 7.83
    if 18.5<f<19.5:
        return 5.67
    if 19.5<f<20.5:
        return 5.08
def cubesat_antenna_RXant_v3_ch1(f):
    if 27.5<f<28.5:
        return 6.77
    if 28.5<f<29.5:
        return 10.53
    if 29.5<f<30.5:
        return 9.62
def DL_link_budget(d,f,g_DLTX,g_DLRX,TXUC_power):
    # d: distance(m)
    # f: frequenct (GHz)
    fspl = 20*np.log10(d)+20*np.log10(f*1e9)+20*np.log10(4*np.pi/3e8)
    if (TXUC_power+g_DLTX)>25:
        DL_CPL=25; print("CPL DL output power saturated")
    TXDC_power = DL_CPL-fspl+g_DLRX; 
    if(TXDC_power)>-4:
        TXDC_power=-4; print("RXDC input power saturated")
    print("TXDC input power: "+str(round(TXDC_power,2))+"dBm")

def UL_link_budget(d,f,g_ULTX,g_ULRX,RXUC_power):
    # d: distance(m)
    # f: frequenct (GHz)
    fspl = 20*np.log10(d)+20*np.log10(f*1e9)+20*np.log10(4*np.pi/3e8)
    if (RXUC_power+g_ULTX)>36:
        UL_UT=36; print("UT UL output power saturated")
    RXDC_power = UL_UT-fspl+g_ULRX
    if(UL_UT-fspl+g_ULRX)>-4:
        RXDC_power=-4; print("RXDC input power saturated")
    print("RXDC input power: "+str(round(RXDC_power,2))+"dBm")
    # return
#-4 dBm

In [29]:
d=3;f=18;TXUC_power=19
g_DLTX= cubesat_antenna_TXant_v3_ch1(f)
g_DLRX= F6212_2x2(f=18,gain_code=27,phicut=0,tx_theta=0,info_show=False)
DL_link_budget(d,f,g_DLTX,g_DLRX,TXUC_power)

CPL DL output power saturated
TXDC input power: -16.17dBm


In [30]:
d=3;f=28;RXUC_power=18
g_ULTX=F5288_2x2(f,gain_code=-2,phicut=0,rx_theta=0,info_show=False)
g_ULRX= cubesat_antenna_RXant_v3_ch1(f)
UL_link_budget(d,f,g_ULTX,g_ULRX,RXUC_power)

UT UL output power saturated
RXDC input power: -28.16dBm


In [128]:
d=3;f=28;RXUC_power=18
g_ULTX=F5288_2x2(f,gain_code=-2,phicut=0,rx_theta=0,info_show=False)
g_ULRX= LPhorn(f)
UL_link_budget(d,f,g_ULTX,g_ULRX,RXUC_power)

RXDC input power: -2.52dBm
